# Search Algorithm Integration

In this tutorial we show to integrate search algorithms in OpenSBT and use them for testing an ADS.

To integrate a search algorithm we need to create a new class that subclasses the `Optimizer` class. There are further three options for the integration:
    
    - 1) If the algorithm exists in `pymoo`, we can instantiate it in the `run` method and assign it to the `algorithm` variable
    - 2) If the algorithm does not exist in pymoo,             
            a) override `run` and implement the algorithm into the run method. The type of the returned object should be a `SimulationResult`.
    -  b) we create a new algorithm by sublassing the `Algorithm` class from pymoo and use approach 1) for integration.

We show in the following in detail for each of the cases how the integration is done.

## Search Algorithm exists in Pymoo

As an example we want to integrate the Particle Swarm Optimization algorithm - wich already exists in [pymoo](https://github.com/anyoptimization/pymoo/blob/main/pymoo/algorithms/soo/nonconvex/pso.py#L130) - into OpenSBT. 

The final PSO related Optimizer class looks as follows:
```python
class PSOOptimizer(Optimizer):

    # This name is used in the output plots 
    algorithm_name = "PSO"

    def __init__(self,
                problem: Problem,
                config: SearchConfiguration):

        log.info("Initialized PSO Optimizer")
        
        # Default code
        self.config = config
        self.problem = problem
        self.res = None

        # The parameters dict is forwarded to the output and is algorithm specific;
        # It contains information about the algorithm configuration
        self.parameters = {
            "Population size" : str(config.population_size),
            "Max number of generations" : str(config.n_generations),
        }

        # Initialize pymoo algorithm
        self.algorithm = PSO(
            pop_size=config.population_size,
        )

        # ((TODO: can push this in optimizer))

        ''' Prioritize max search time over set maximal number of generations'''
        if config.maximal_execution_time is not None:
            self.termination = get_termination("time", config.maximal_execution_time)
        else:
            self.termination = get_termination("n_gen", config.n_generations)

        # ((Do we need that?))
        self.save_history = True
```

Registration of algorithm (TODO)

Adding to run (TODO)

## Search Algorithm does not exist in Pymoo

((TODO)) NSGAII-DT example

1. User needs to create a results object
2. Else?







